should be nice and simple

In [ ]:
import pandas as pd
import psycopg2
import os
import uuid
import logging
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables from .env file
load_dotenv()

# Connection parameters
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")

# Connect to the database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cursor = conn.cursor()

countries_dict = {}

cursor.execute("SELECT country_id, name, iso_alpha3 FROM countries")

data = cursor.fetchall()
for record in data:
   print(record)
   countries_dict[record[0]] = record[1]
   # logger.info(f"Fetched country records")

# Close the cursor and connection
cursor.close()
conn.close()

# Print the dictionary
# print(countries_dict)

# Convert dictionary to DataFrame
countries_df = pd.DataFrame(data, columns=['country_id', 'name', 'iso_alpha3'])

# Write DataFrame to CSV
countries_df.to_csv('../data/countries_biodiversity/countries_nile.csv', index=False)

In [3]:
# Load countries_species.csv into a DataFrame
# counts_df = pd.read_csv('../data/countries_species_counts.csv')
counts_df = pd.read_csv('../data/countries_biodiversity_rankings.csv')

# Merge df with countries_species_df on the 'binominal_name' column
merged_df = pd.merge(countries_df, counts_df, left_on='name', right_on='Country', how='inner')

merged_df.to_csv('../data/countries_biodiversity/merged_countries_biodiversity_rankings.csv', index=False)

In [4]:
# Path to your CSV file
csv_file_path = '../data/countries_biodiversity/merged_countries_biodiversity_rankings.csv'

# Read CSV file into DataFrame
df = pd.read_csv(csv_file_path)

# Convert any non-numeric data that might be in numeric columns to NaN, then to None
numeric_columns = ['Amphibians', 'Birds', 'Fish', 'Mammals', 'Reptiles', 'Plants']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.where(pd.notnull(df), None)

# Calculate ranks for each category
for col in numeric_columns:
    rank_col = f"{col.lower()}_rank"
    df[rank_col] = df[col].rank(method='min', ascending=False).astype('Int64')

# categories = ['Amphibians', 'Birds', 'Reptiles', 'Fish', 'Mammals', 'Plants']

# for category in categories:
#     percentile_col_name = f"{category} Percentile"
#     df[percentile_col_name] = df[category].rank(pct=True) * 100
#     df[percentile_col_name] = df[percentile_col_name].round(2)

display(df)

,country_id,name,iso_alpha3,Biodiversity Rank,Country,Amphibians,Birds,Fish,Mammals,Reptiles,Plants,amphibians_rank,birds_rank,fish_rank,mammals_rank,reptiles_rank,plants_rank
0,0980bef2-5fa2-46a4-a416-66e91d7524fe,Yemen,YEM,104,Yemen,6.0,421.0,784.0,91.0,117.0,2900.0,148,94,20,117,82,98
1,606e8692-b533-4311-b98a-abcce7a4938b,Afghanistan,AFG,98,Afghanistan,12.0,507.0,128.0,135.0,116.0,5000.0,128,65,129,84,84,55
2,f099b01b-ac4a-4a7f-8fd4-dd9e1a6fef3d,Albania,ALB,140,Albania,17.0,317.0,452.0,92.0,41.0,2450.0,111,132,73,114,127,116
3,86c08a49-0099-4fce-8515-ca15e0c80edb,Algeria,DZA,111,Algeria,16.0,388.0,511.0,109.0,111.0,3139.0,116,102,60,99,85,89
4,85d0b09c-5f52-4f64-a22a-d9e7c4c68f9b,American Samoa,ASM,208,American Samoa,NaN,68.0,768.0,21.0,4.0,NaN,<NA>,197,21,202,202,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,face4f7f-36c1-44bc-afc1-fc5ae6dd0490,Virgin Islands (U.S.),VIR,180,Virgin Islands (U.S.),7.0,244.0,577.0,31.0,44.0,748.0,144,155,48,178,124,169
222,d2872c58-a585-436f-a3c7-ec8fa8df15f2,Wallis and Futuna,WLF,228,Wallis and Futuna,NaN,59.0,208.0,0.0,7.0,NaN,<NA>,199,116,215,191,<NA>
223,038dee07-69de-4f98-927e-b824f22542b8,Western Sahara,ESH,199,Western Sahara,2.0,197.0,448.0,0.0,66.0,NaN,176,168,74,215,111,<NA>
224,69b434e4-f25f-400e-9a4a-82dcee6e65ef,Zambia,ZMB,48,Zambia,91.0,772.0,454.0,260.0,191.0,3309.0,39,17,72,34,49,85


In [18]:
import pandas as pd
import numpy as np  # Import NumPy
import psycopg2
import uuid
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Database connection parameters
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")

# Path to your CSV file
csv_file_path = '../data/countries_biodiversity/merged_countries_biodiversity_rankings.csv'

# Read CSV file into DataFrame
df = pd.read_csv(csv_file_path)

# Clean numeric columns by removing commas and converting to numeric type
numeric_columns = ['Amphibians', 'Birds', 'Fish', 'Mammals', 'Reptiles', 'Plants']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

# Calculate ranks for each biodiversity category
for col in numeric_columns:
    rank_col = f"{col.lower()}_rank"
    df[rank_col] = df[col].rank(ascending=False, method='min').astype('Int64')

# Explicitly handle missing values, converting pd.NA, np.nan to None
df.replace({pd.NA: None, np.nan: None}, inplace=True)

# Adding UUIDs for primary key and timestamps
df['country_biodiversity_rank_id'] = [str(uuid.uuid4()) for _ in range(len(df))]
current_time = datetime.now()
df['created'] = current_time
df['updated'] = current_time

# Define the SQL INSERT statement
insert_sql = """
INSERT INTO countries_biodiversity_rankings
(country_biodiversity_rank_id, country_id, biodiversity_rank, amphibians, amphibians_rank, birds, birds_rank,
fish, fish_rank, mammals, mammals_rank, reptiles, reptiles_rank, plants, plants_rank, created, updated)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Database insertion
try:
    with psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port) as conn:
        with conn.cursor() as cursor:
            # Iterate through DataFrame rows for insertion
            for index, row in df.iterrows():
                data_tuple = (
                    row['country_biodiversity_rank_id'], row['country_id'], row.get('Biodiversity Rank', None),
                    row['Amphibians'], row.get('amphibians_rank', None),
                    row['Birds'], row.get('birds_rank', None),
                    row['Fish'], row.get('fish_rank', None),
                    row['Mammals'], row.get('mammals_rank', None),
                    row['Reptiles'], row.get('reptiles_rank', None),
                    row['Plants'], row.get('plants_rank', None),
                    row['created'], row['updated']
                )
                cursor.execute(insert_sql, data_tuple)
        conn.commit()
    print("All records inserted successfully")
except Exception as e:
    print(f"Error during database insertion: {e}")
finally:
    if conn:
        cursor.close()
        conn.close()


All records inserted successfully
